After observing the yahoo finance package, found some days' closing price are not correct. So, the extra script help to fix the data. There is no need to download and fix manully everytime. I already email to the yahoo finance to fix this data. If they fix, then there will be no use of this first script in future.


In [10]:
import yfinance as yf
import pandas as pd

def download_and_fix_eurusd_close():
    # Download EUR/USD data
    print("Downloading EUR/USD data...")
    ticker = "EURUSD=X"
    df = yf.download(ticker, start="2000-01-01", progress=False)

    df = df[['Close']].copy()

    # Flatten columns if multi-index (common in yfinance)
    if isinstance(df.columns, pd.MultiIndex):
        print("Flattening columns...")
        df.columns = df.columns.get_level_values(0)

    # Taken from https://www.kaggle.com/datasets/lehenzehra/eurusd-daily-data-ohlc?select=EURUSD_D1_Sorted.csv
    corrections = {
        "2008-01-08": 1.4705,
        "2008-02-08": 1.4503,
        "2008-08-08": 1.5074,
        "2008-09-08": 1.4250,
        "2008-10-08": 1.3650,
        "2008-12-08": 1.2930,
    }

    print("\nApplying corrections...")

    # Apply corrections
    for date_str, price in corrections.items():
        dt = pd.Timestamp(date_str)
        if dt in df.index:
            print(f"Fixing {date_str}: Old={df.at[dt, 'Close']:.4f} -> New={price:.4f}")
            df.at[dt, 'Close'] = price
        else:
            print(f"Warning: {date_str} not found in data.")


    # Save to CSV
    output_file = "EURUSD_Close_Fixed.csv"
    df.to_csv(output_file)
    print(f"\nDone. Saved to {output_file}")


download_and_fix_eurusd_close()

/tmp/ipython-input-2381738963.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2000-01-01", progress=False)


Flattening columns...

Applying corrections...
Fixing 2008-01-08: Old=1.5571 -> New=1.4705
Fixing 2008-02-08: Old=1.5571 -> New=1.4503
Fixing 2008-08-08: Old=1.5049 -> New=1.5074
Fixing 2008-09-08: Old=1.5050 -> New=1.4250
Fixing 2008-10-08: Old=1.4957 -> New=1.3650
Fixing 2008-12-08: Old=1.4918 -> New=1.2930

Done. Saved to EURUSD_Close_Fixed.csv


Load the corrected data and make a 'difference' percentafge column based on the closing price.

In [11]:
df = pd.read_csv("EURUSD_Close_Fixed.csv", index_col=0, parse_dates=True)
df['difference'] = df['Close'].pct_change() * 100
df.head()
df.to_csv('EURUSD_Close_Fixed_with_difference.csv')


The main script started from here. Creating main data table for calculating Euclidean Distane.

In [12]:
data = df['difference'].dropna().tolist()

main_data_rows = []
window_size = 6

# Iterate through the data to create feature vectors and target values
for i in range(len(data) - window_size + 1):
    row = data[i : i + window_size]
    feature_vector = row[:5]
    true_value = row[5]
    main_data_rows.append(feature_vector + [true_value])

# Create column names for the new DataFrame
column_names = [f'p{j+1}' for j in range(5)] + ['true_value_next_day']

# Create the 'main_data' DataFrame
main_data = pd.DataFrame(main_data_rows, columns=column_names)

# Set the DataFrame index to start from 1
main_data.index = range(1, len(main_data) + 1)

#Saving "Main Data Table"
main_data.to_csv('main_data_table.csv', index_label='Vectors')
print("Saved 'main_data_table.csv'.")

Saved 'main_data_table.csv'.


After forming the "difference" column as vectors, calcultaing the Euclidean Distance. And do statics of the positive and negative outcomes.

In [33]:
import numpy as np

#Create base_vector range -2 to 2, each time take 5 datapoints for a vectors.
generated_base_vectors = np.random.uniform(low=-2.0, high=2.0, size=(10, 5))


all_results = []

for current_threshold in np.arange(0.5, 1.51, 0.1):
    print(f"\nCalculating ED for T = {current_threshold:.1f}...")

    # Iterate through each generated base vector
    for gen_base_vec_idx in range(generated_base_vectors.shape[0]):
        current_base_np_vector = generated_base_vectors[gen_base_vec_idx]
        accepted_forecasts_for_this_base_vector = []
        euclidean_distance = 0

        for index, row in main_data.iterrows():
            feature_vector = np.array(row[['p1', 'p2', 'p3', 'p4', 'p5']].tolist())
            euclidean_distance = np.linalg.norm(feature_vector - current_base_np_vector)

            if euclidean_distance < current_threshold:
                accepted_forecasts_for_this_base_vector.append(row['true_value_next_day'])

        accepted_series_for_this_base_vector = pd.Series(accepted_forecasts_for_this_base_vector)

        positive_outcomes = (accepted_series_for_this_base_vector > 0).sum()
        negative_outcomes = (accepted_series_for_this_base_vector <= 0).sum()
        total_accepted_forecasts = len(accepted_series_for_this_base_vector)

        all_results.append({
          'THRESHOLD': f"{current_threshold:.1f}",
          'base_vector': current_base_np_vector,
          'ED': euclidean_distance,
          'accepted_forecast': accepted_forecasts_for_this_base_vector,
          'Total Accepted Forecasts': total_accepted_forecasts,
          'Positive Outcomes': f'{(positive_outcomes/total_accepted_forecasts)*100:.2f}' if total_accepted_forecasts > 0 else 0,
          'Negative Outcomes': f'{(negative_outcomes/total_accepted_forecasts)*100:.2f}' if total_accepted_forecasts > 0 else 0,
      })

result_df = pd.DataFrame(all_results)
print("\nResults Different THRESHOLDS:")
display(result_df)
result_df.to_csv('overall_reuslt.csv', index= True)



Calculating ED for T = 0.5...

Calculating ED for T = 0.6...

Calculating ED for T = 0.7...

Calculating ED for T = 0.8...

Calculating ED for T = 0.9...

Calculating ED for T = 1.0...

Calculating ED for T = 1.1...

Calculating ED for T = 1.2...

Calculating ED for T = 1.3...

Calculating ED for T = 1.4...

Calculating ED for T = 1.5...

Results Different THRESHOLDS:


,THRESHOLD,base_vector,ED,accepted_forecast,Total Accepted Forecasts,Positive Outcomes,Negative Outcomes
0,0.5,"[-1.2048520195478938, 1.3202560710326843, -1.2...",3.080051,[],0,0,0
1,0.5,"[-0.2276623442467569, -1.0758489208874664, -1....",2.643784,[],0,0,0
2,0.5,"[1.5863527876968742, 1.8746317337939398, -1.27...",3.557689,[],0,0,0
3,0.5,"[-1.1785523990355635, -1.5251592963976335, -0....",2.428594,[1.1119700690889855],1,100.00,0.00
4,0.5,"[-0.048773162328365594, 1.3095979782640321, -0...",1.918876,"[-1.6691137667364075, 0.40043300349124067]",2,50.00,50.00
...,...,...,...,...,...,...,...
105,1.5,"[0.5355354279106344, -0.24934404488187134, 1.6...",2.287920,"[0.27919443535306954, 0.16751263940757433, -0....",50,46.00,54.00
106,1.5,"[-0.5177639228806812, -0.511194308228812, 1.35...",2.335135,"[0.46755684402493003, -1.6856497093415745, -0....",87,51.72,48.28
107,1.5,"[-1.1027779500469856, 0.6046142114993529, -1.2...",2.713663,"[-0.27755080928901776, 0.8494937769757405, -0....",54,66.67,33.33
108,1.5,"[-0.6432559268451064, -0.4729033290989584, 0.2...",2.491250,"[-0.1072722832722861, -0.5763091688564037, -0....",52,48.08,51.92
